##### Mapping dataclasses using Declarative With Imperative Table

An example of a mapping using `@dataclass` using `Declarative with Imperative Table` (a.k.a. _Hybrid Declarative_) is below. A __complete `Table` object__ is __constructed explicitly__ and assigned to the `__table__` attribute. _Instance fields_ are defined using _normal dataclass syntaxes_. Additional `MapperProperty` definitions such as `relationship()`, are placed in the `__mapper_args__` class-level dictionary underneath the properties key, corresponding to the `mapper.properties` parameter.

In [1]:
from typing import List, Optional
from __future__ import annotations
from dataclasses import dataclass, field

from sqlalchemy import Column, ForeignKey, Integer, String, Table
from sqlalchemy.orm import registry, relationship

In [2]:
mapper_registry = registry()

In [3]:
@mapper_registry.mapped
@dataclass
class User:
    __table__ = Table(
        "user",
        mapper_registry.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String(50)),
        Column("fullname", String(50)),
        Column("nickname", String(12)),
    )
    
    id: int = field(init=False)
    name: Optional[str] = None
    fullname: Optional[str] = None
    nickname: Optional[str] = None
    addresses: List[Address] = field(default_factory=list)
    
    __mapper_args__ = {
        "properties": {"addresses": relationship("Address")},
    }

In [4]:
@mapper_registry.mapped
@dataclass
class Address:
    __table__ = Table(
        "address",
        mapper_registry.metadata,
        Column("id", Integer, primary_key=True),
        Column("user_id", Integer, ForeignKey("user.id")),
        Column("email_address", String(50)),
    )
    
    id: int = field(init=False)
    user_id: int = field(init=False)
    email_address: Optional[str] = None

In the above example, the `User.id`, `Address.id`, and `Address.user_id` attributes are defined as `field(init=False)`. This means that _parameters for these_ __won't be added to `__init__()` methods__, but `Session` will still be able to __set them after getting their values during flush__ from _autoincrement_ or _other default value generator_. To allow them to be _specified in the constructor explicitly_, they would instead be __given a default value of None__.

For a `relationship()` to be _declared separately_, it needs to be __specified directly__ within the `mapper.properties` dictionary which itself is specified within the `__mapper_args__` dictionary, so that it is passed to the `mapper()` construction function. An alternative to this approach is in the next example.